回帰分析（入力変数を与えたときの出力変数値を求めるモデル）による因果推論

回帰モデルの出力変数には因果効果が表れる変数（結果変数）を使用する．  
結果変数がカテゴリ変数である場合は，ロジスティック回帰を使用する．

モデルの入力変数は観測可能な全変数ではなく，バックドアパスを閉じるように因果ダイアグラムをd分離した後に残っている変数のみを使用する（d分離）．

テレビCMの広告効果の推定を例に実装  
よってCMをみたかどうかがZ（原因変数）

結果変数Yは購入量とする

入力変数について，変数Zと変数Yの間に擬似相関を生む交絡因子（共通因子）を共変量として使う．

In [1]:
# 乱数シード
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

In [3]:
# 標準正規分布の生成用
from numpy.random import *
# 正規化関数
import scipy.stats
# シグモイド関数
from scipy.special import expit


import pandas as pd 
import matplotlib.pyplot as plt

# 自前でデータ作成

In [7]:
num_data = 200

入力変数には，d分離した後で因果ダイアグラムに残っている変数を使う．これはd分離後残っている変数はバックドアパスを生むため，線形回帰で閉じる必要がある．

## 性別，年齢

In [6]:
#一様乱数
# age
x_1 = randint(15, 76, num_data) 
# 性別（0: 女性，1: 男性）
x_2 = randint(0, 2, num_data)

## テレビCMをみたか

In [13]:
# ノイズの生成
e_z = randn(num_data)

# シグモイド関数に入れる線形関数・・・（1）
z_base = x_1 + (1 - x_2)*10 - 40 + 5*e_z

# シグモイド関数の計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうか（0: 見ていない，1: 見た）
Z = np.array([])

# （1）より年配の女性の方が見やすくなる
for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)

## 購入量Y

In [14]:
# ノイズの生成
e_y = randn(num_data)

# 若年層で，男性で，テレビを見ていると購入が多くなる
Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y 

# 作成データ

In [15]:
df = pd.DataFrame({
    'age': x_1,
    'sex': x_2,
    'cm': Z,
    'amount': Y
})

In [16]:
df.head()

,age,sex,cm,amount
0,61,1,1.0,60.624920
1,26,0,0.0,48.618056
2,21,0,1.0,70.781316
3,38,1,1.0,85.292791
4,33,1,1.0,86.680170


## CM別平均

In [17]:
print(df[df['cm']==1.0].mean())
print('------')
print(df[df['cm']==0.0].mean())

age       53.259259
sex        0.577778
cm         1.000000
amount    53.988896
dtype: float64
------
age       32.600000
sex        0.600000
cm         0.000000
amount    65.327939
dtype: float64


# 回帰分析

In [18]:
from sklearn.linear_model import LinearRegression

# 説明変数
X = df[['age', 'sex', 'cm']]

# 目的変数
y = df['amount']

# 回帰
reg = LinearRegression().fit(X, y)

# 回帰の係数
print('coef: ', reg.coef_)

coef:  [-1.05642502 30.53240429 11.16441373]


CMを見たことによる購入量の増加は11.1と推定されるため，CMの介入効果は11.1となる（ちなみに本例は自作データなのでその係数に近くなる）

ATE, ATT, ATUは  
回帰モデルなのでZの係数そのまま11.1